# Appendix 

Here we document how we generated the data for the fleet assignment problem

In [19]:
from matplotlib import collections  as mc
import pylab as pl
import matplotlib.pyplot as plt
import numpy as np
import random
import itertools
import pandas as pd

airps=5 #arbitrary number of airports
fltypes=5 #five types of fleet
fleet=[random.randint(1,5),random.randint(1,5),random.randint(1,5),random.randint(1,5),random.randint(1,5)] #random number of each type

ct is our randomly generated operation cost for every (fleet,airport,airport) set. opcost is 0 if the airports are the same.

In [20]:
ct=[]
for i in range(5):
    for j in range(5):
        for k in range(5):
            if j!=k:
                ct.append([i+1,j+1,k+1,random.randint(5,20)])
            else:
                ct.append([i+1,j+1,k+1,0])

In [21]:
[random.randint(1,5),random.randint(1,5),random.randint(1,5),random.randint(1,5)] #random number of flights for n-1 airports

[2, 4, 1, 4]

We use the following code to generate number of flights from the final airport and balance our data. 
Number of incoming and outgoing flights at every airport is balanced through these constraints to generate flights between the airports.

In [22]:
%load_ext gams_magic
import pandas as pd
%gams_cleanup -k
%gams_reset

The gams_magic extension is already loaded. To reload it, use:
  %reload_ext gams_magic


In [23]:
%%gams
set s/1*5/;
parameter sums(s)/1 5,2 3,3 5,4 2/;
alias(t,s);
integer variable x(s,t),summ(s);
free variable fv;
equations e1,e2,e3,e4,e5,e6;
summ.fx(s)=sums(s)$(ord(s)<>5);
x.fx(s,s)=0;

e1..
fv=g=0;

e2(s)..
sum(t,x(s,t))=e=sum(t,x(t,s));

e3(s,t)..
x(s,t)=g=0;

e4(s)$(ord(s)<>5)..
sum(t,x(s,t))=e=summ(s);

e5..
sum(t,x('5',t))=e=fv;

e6(t,s)..
x(t,s)=l=2;

model net /all/;
solve net using mip minimizing fv;

,Solver Status,Model Status,Objective,#equ,#var,Model Type,Solver,Solver Time
0,Normal (1),Optimal Global (1),1.0,61,30,MIP,CPLEX,0.026


In [24]:
%gams_pull -d x
xx=x.loc[x['level']>0]
xx=xx.drop(columns=['marginal','lower','upper','scale']).reset_index().drop(['index'],axis=1)
xx.columns=['air1','air2','flights']

In [25]:
times2=[[]]*len(times)
for i in range(len(times)):
    dummy=[]
    for j in range(len(times[i])//2):
        dummy.append([times[i][2*j],times[i][2*j+1]])
        times2[i]=dummy

In [26]:
flightsch=[]
for i in range(len(xx)):
    for j in range(int(xx.loc[i].flights)):
        flightsch.append([xx.loc[i].air1,xx.loc[i].air2,times2[i][j],[50*random.randint(2,8),50*random.randint(2,6)]])
for i in range(len(flightsch)):
    flightsch[i]=list(itertools.chain(*flightsch[i]))

A flight schedule of the format [Airport1, Airport2, time slot 1, time slot 2, fare, demand] is generated

In [27]:
display(flightsch)

[['1', '2', 6, 10, 250, 200],
 ['1', '3', 2, 6, 150, 200],
 ['1', '3', 9, 15, 100, 100],
 ['1', '4', 7, 11, 100, 250],
 ['1', '5', 1, 6, 300, 200],
 ['2', '1', 7, 11, 200, 150],
 ['2', '1', 13, 15, 300, 200],
 ['2', '3', 2, 3, 250, 300],
 ['3', '1', 1, 4, 150, 250],
 ['3', '1', 9, 12, 350, 300],
 ['3', '2', 7, 8, 150, 100],
 ['3', '2', 9, 14, 300, 300],
 ['3', '4', 12, 13, 100, 150],
 ['4', '3', 10, 13, 400, 200],
 ['4', '3', 14, 16, 300, 250],
 ['5', '1', 3, 15, 300, 200]]

In [28]:
schedule=pd.DataFrame(flightsch,columns=['air','air','ts','ts','fare','demand'])
farev=schedule.drop(columns=['demand'])
demv=schedule.drop(columns=['fare'])
#generating dataframes for demand and fare

In [29]:
skyr=demv
for i in range(len(skyr)):
    skyr.loc[i,'demand']=1
skyr.columns=['air','air','ts','ts','arcs']
#generating "arc" variable values for valid flights

In [30]:
opcosts=pd.DataFrame(ct)

In [31]:
demv.to_excel("demand.xlsx")
farev.to_excel("fares.xlsx")
skyr.to_excel("skarc.xlsx")
opcosts.to_excel("opcos.xlsx")

Since GDXXRW doesn't work with GAMS Magic, we have to generate the gdx files externally on GAMS studio.
<br>This requires the following code:
```
    $call gdxxrw demand.xlsx maxDupeErrors=20 par=dem rng=Sheet1!B2:F19 rDim=4
    $call gdxxrw fares.xlsx maxDupeErrors=20 par=fare rng=Sheet1!B2:F19 rDim=4
    $call gdxxrw skarc.xlsx maxDupeErrors=20 par=skyarcs rng=Sheet1!B2:F19 rDim=4
    $call gdxxrw opcos.xlsx maxDupeErrors=20000 par=opcos rng=Sheet1!B2:E126 rDim=3
```